#### Heroku Database Dataloading

This script is the code used for loading our open data sources into our Heroku cloud database. 
Initially, we planned to load all data sources into the cloud and wrangle our data via SQL, in the spirit of utilising a relational database. However, due to database constraints (10k row max), for iteration 1 the cleaning and wrangling has predominantly taken place using python pandas before it has been loaded to the database as the final table for our intended function (iteration 1 - user story 1.3). 

Moving into future iterations we may explore other options that have fewer constraints and allow us to load more data and tables. 

#### Import Libraries

- Psycopg2: for database connection
- Pandas: for data cleaning/wrangling
- SQLAlchemy: for sql usage with database

In [2]:
import psycopg2
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import inspect

#### Connection to database

Have removed this code block for now, to keep connection confidential

In [2]:
# create engine, connection to database
engine = create_engine(#connection url)

#### Import Data

Import open data sources (supplied from VicRoads/Department of Transport via CSVs)
The data is nearly 20 years worth of police reported Victorian road crashes, the data is at an accident level. 
Accident table describes main details of the accident, the node detailing the precise location and the vehicle table providing detailed information on vehicles involved in their respective accidents.

In [7]:
# low memory = False due to mixed dtype in some of the open data sources

vehicles = pd.read_csv("VEHICLE.csv", low_memory = False)
accident = pd.read_csv("ACCIDENT.csv", low_memory = False)
node = pd.read_csv("NODE.csv")

##### Clean up datasets

Began cleaning/wrangling our datasets, started with vehicles as currently we have all traffic accidents and are only interested in bicycle related accidents, the vehicle table will help us filter done to bicycle only. 

In [5]:
vehicles.head()

,ACCIDENT_NO,VEHICLE_ID,VEHICLE_YEAR_MANUF,VEHICLE_DCA_CODE,INITIAL_DIRECTION,ROAD_SURFACE_TYPE,Road Surface Type Desc,REG_STATE,VEHICLE_BODY_STYLE,VEHICLE_MAKE,...,VEHICLE_COLOUR_1,VEHICLE_COLOUR_2,CAUGHT_FIRE,INITIAL_IMPACT,LAMPS,LEVEL_OF_DAMAGE,OWNER_POSTCODE,TOWED_AWAY_FLAG,TRAFFIC_CONTROL,Traffic Control Desc
0,T20060000010,A,1996.0,2,SW,1,Paved,V,SEDAN,MITSUB,...,MRN,ZZ,2,F,2,3,3130.0,1,1,Stop-go lights
1,T20060000010,B,2003.0,1,NW,1,Paved,V,COUPE,UNKN,...,BLU,ZZ,2,2,2,3,3977.0,1,1,Stop-go lights
2,T20060000010,C,2001.0,8,NW,1,Paved,V,SEDAN,FORD,...,YLW,ZZ,2,F,2,2,3804.0,2,1,Stop-go lights
3,T20060000018,A,1998.0,2,S,1,Paved,V,DC UTE,TOYOTA,...,GRY,ZZ,2,9,0,3,3175.0,1,0,No control
4,T20060000018,B,1991.0,1,N,1,Paved,V,SEDAN,SUBARU,...,BLU,ZZ,2,F,0,5,3805.0,1,11,Giveway sign


In [4]:
# review necessary columns
vehicles.columns

Index(['ACCIDENT_NO', 'VEHICLE_ID', 'VEHICLE_YEAR_MANUF', 'VEHICLE_DCA_CODE',
       'INITIAL_DIRECTION', 'ROAD_SURFACE_TYPE', 'Road Surface Type Desc',
       'REG_STATE', 'VEHICLE_BODY_STYLE', 'VEHICLE_MAKE', 'VEHICLE_MODEL',
       'VEHICLE_POWER', 'VEHICLE_TYPE', 'Vehicle Type Desc', 'VEHICLE_WEIGHT',
       'CONSTRUCTION_TYPE', 'FUEL_TYPE', 'NO_OF_WHEELS', 'NO_OF_CYLINDERS',
       'SEATING_CAPACITY', 'TARE_WEIGHT', 'TOTAL_NO_OCCUPANTS',
       'CARRY_CAPACITY', 'CUBIC_CAPACITY', 'FINAL_DIRECTION', 'DRIVER_INTENT',
       'VEHICLE_MOVEMENT', 'TRAILER_TYPE', 'VEHICLE_COLOUR_1',
       'VEHICLE_COLOUR_2', 'CAUGHT_FIRE', 'INITIAL_IMPACT', 'LAMPS',
       'LEVEL_OF_DAMAGE', 'OWNER_POSTCODE', 'TOWED_AWAY_FLAG',
       'TRAFFIC_CONTROL', 'Traffic Control Desc'],
      dtype='object')

In [8]:
# Filter and cut the vehicles data
# only interested in bikes, and more interested in accidents rather than actual bike so only taking 3 fields
vehicles = vehicles[vehicles['Vehicle Type Desc'] == 'Bicycle']
vehicles = vehicles[['ACCIDENT_NO', 'VEHICLE_ID', 'Vehicle Type Desc']]

In [13]:
vehicles.shape

(20994, 3)

In [9]:
# similar preparation for accidents
accident.columns

Index(['ACCIDENT_NO', 'ACCIDENTDATE', 'ACCIDENTTIME', 'ACCIDENT_TYPE',
       'Accident Type Desc', 'DAY_OF_WEEK', 'Day Week Description', 'DCA_CODE',
       'DCA Description', 'DIRECTORY', 'EDITION', 'PAGE', 'GRID_REFERENCE_X',
       'GRID_REFERENCE_Y', 'LIGHT_CONDITION', 'Light Condition Desc',
       'NODE_ID', 'NO_OF_VEHICLES', 'NO_PERSONS', 'NO_PERSONS_INJ_2',
       'NO_PERSONS_INJ_3', 'NO_PERSONS_KILLED', 'NO_PERSONS_NOT_INJ',
       'POLICE_ATTEND', 'ROAD_GEOMETRY', 'Road Geometry Desc', 'SEVERITY',
       'SPEED_ZONE'],
      dtype='object')

In [10]:
accident = accident[['ACCIDENT_NO', 'ACCIDENTDATE', 'ACCIDENTTIME', 'Accident Type Desc'
                    , 'Day Week Description', 'NODE_ID', 'SEVERITY', 'SPEED_ZONE']]

In [11]:
#evaluate dtypes
accident.dtypes

ACCIDENT_NO             object
ACCIDENTDATE            object
ACCIDENTTIME            object
Accident Type Desc      object
Day Week Description    object
NODE_ID                  int64
SEVERITY                 int64
SPEED_ZONE               int64
dtype: object

In [14]:
accident['ACCIDENTDATE'] = pd.to_datetime(accident['ACCIDENTDATE'], format = '%d/%m/%Y')

In [15]:
# due to database constraints (10k) and bike accidents ~21k, will filter down by most recent years
# create year column for filtering
accident['Year'] = accident['ACCIDENTDATE'].dt.year

In [16]:
accident['Year'].value_counts()

2016    15852
2015    15748
2014    14475
2019    14164
2008    14128
2011    14091
2013    14009
2012    13913
2007    13854
2009    13851
2006    13697
2010    13676
2017    13393
2018    12797
2020     6060
Name: Year, dtype: int64

In [17]:
# create temporary df that combines accident data with bicycle data 
# essentially filtering accidents to bike accidents, hence using inner join 
temp = accident.merge(vehicles, on='ACCIDENT_NO', how='inner')

In [18]:
temp.shape

(20994, 11)

In [19]:
# check unique accident nos, 1 accident can have mulitple vehicles or in our case multiple bicycles
# hence expect this to be slightly lower 

temp['ACCIDENT_NO'].nunique()

20450

In [20]:
temp['Year'].value_counts()

2014    1626
2015    1606
2011    1586
2013    1572
2010    1500
2009    1496
2016    1441
2012    1439
2008    1388
2019    1381
2017    1370
2007    1319
2018    1302
2006    1267
2020     701
Name: Year, dtype: int64

In [21]:
# Around 1.5k per year, will use 2015 onwards to meet db constraints but still maintain relevant data 
temp = temp[temp['Year'] >= 2015]

In [22]:
temp.shape

(7801, 11)

In [23]:
node.shape

(221797, 12)

In [ ]:
# append geo-coordinates to data 
# this will be the data loaded into database for iteration 1

# left join here, all accidents should have a location, but if unknown still want accident details

final = temp.merge(node, on='ACCIDENT_NO', how = 'left')

In [ ]:
final.head()

In [ ]:
final.columns

In [ ]:
# final filter 

final = final[['ACCIDENT_NO', 'ACCIDENTDATE', 'ACCIDENTTIME', 'Accident Type Desc',
       'Day Week Description', 'NODE_ID_x', 'SEVERITY', 'SPEED_ZONE', 'Year',
       'VEHICLE_ID', 'Vehicle Type Desc','LGA_NAME', 'LGA_NAME_ALL', 'REGION_NAME',
       'Lat', 'Long', 'POSTCODE_NO']]

In [ ]:
final.dtypes

In [ ]:
final.head()

In [ ]:
# Load df to database as table 'accidents'

# Suvansh said got notified at 6am, I kicked this off at 11pm, hopefully doesn't take this long normally 

final.to_sql('accidents', con = engine, if_exists='append')

In [12]:
# Quick test 

print(engine.execute(
    """
    SELECT *
    FROM
        accidents
    LIMIT 5;
    """).fetchall())


[(0, 'T20150000067', datetime.datetime(2015, 1, 1, 0, 0), '18:10:00                      ', 'Collision with vehicle', 'Thursday', 280320, 3, 777, 2015, 'A', 'Bicycle', 'GEELONG', 'GEELONG', 'SOUTH WESTERN REGION', -38.11077182, 144.6419289, 3223.0), (1, 'T20150000093', datetime.datetime(2015, 1, 2, 0, 0), '12:05:00                      ', 'No collision and no object struck', 'Friday', 36940, 3, 60, 2015, 'A', 'Bicycle', 'MELBOURNE', 'MELBOURNE', 'METROPOLITAN NORTH WEST REGION', -37.81254531, 144.9651824, 3000.0), (2, 'T20150000096', datetime.datetime(2015, 1, 2, 0, 0), '16:55:00                      ', 'Collision with vehicle', 'Friday', 281773, 3, 60, 2015, 'A', 'Bicycle', 'MELBOURNE', 'MELBOURNE,PORT PHILLIP', 'METROPOLITAN NORTH WEST REGION', -37.84314893, 144.97806469999998, 3004.0), (3, 'T20150000156', datetime.datetime(2015, 1, 2, 0, 0), '21:00:00                      ', 'Vehicle overturned (no collision)', 'Friday', 280598, 3, 60, 2015, 'A', 'Bicycle', 'WHITEHORSE', 'WHITEHORSE

In [11]:
# Inspect table metadata so can produce ERD/data dictionary for others 

inspector = inspect(engine)
inspector.get_columns('accidents')

[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'ACCIDENT_NO',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'ACCIDENTDATE',
  'type': TIMESTAMP(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'ACCIDENTTIME',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Accident Type Desc',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Day Week Description',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'NODE_ID_x',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'SEVERITY',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'a

The intended application of this open data for app is to pin accident locations onto a map, however this may be too ambitious in iteration 1, if time constrained will simply show dataviz/table to convey locations of accidents. 

Following produces summary table by LGA (suburb) the total accidents in our database (from 2015 onwards). 

In [23]:
with engine.connect() as con:

    rs = con.execute("""
    CREATE TABLE lga_summary AS
    SELECT 
        "LGA_NAME"
        , COUNT(DISTINCT("ACCIDENT_NO")) AS no_accidents
    FROM accidents
    GROUP BY "LGA_NAME"
    ORDER BY no_accidents DESC;
    
    """)


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [4]:
# some test tables still in db try to see what they are 
print(engine.execute(
    """
    SELECT *
    FROM INFORMATION_SCHEMA.TABLES;
    """).fetchall())

[('d4jhuntcv1di7g', 'public', 'vehicles', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('d4jhuntcv1di7g', 'public', 'master_data', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('d4jhuntcv1di7g', 'public', 'accidents', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('d4jhuntcv1di7g', 'public', 'test_vehicles', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('d4jhuntcv1di7g', 'public', 'lga_summary', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('d4jhuntcv1di7g', 'pg_catalog', 'pg_type', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('d4jhuntcv1di7g', 'pg_catalog', 'pg_foreign_table', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('d4jhuntcv1di7g', 'pg_catalog', 'pg_roles', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('d4jhuntcv1di7g', 'pg_catalog', 'pg_settings', 'VIEW', None, None, None, None, None, 'NO', 'NO', None), ('d4jhuntcv1di7g', 'pg_catalog

In [31]:
# summary table details
inspector = inspect(engine)
inspector.get_columns('lga_summary')

[{'name': 'LGA_NAME',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'no_accidents',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

In [5]:
# dropping test tables 
# engine.execute("DROP TABLE test_vehicles")